In [ ]:
import os
seed = 40
os.environ['PYTHONHASHSEED']=str(seed)

import random
import seaborn as sns
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import *
style.use('ggplot')
import pandas as pd
import numpy as np
import time
import torch.profiler
import torch.autograd.profiler as profiler
from scipy import stats as st
import sklearn.preprocessing as preprocess
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import PolynomialFeatures
import torch.optim as optim
import optuna



random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Context Integrated RNN - CiRNN

In [ ]:
# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class ContextGRU(torch.nn.Module):
    """
     simple GRU cell network
    """

    def __init__(self, input_dim, hidden_dim, output_dim, context_dim):
        super(ContextGRU, self).__init__()

        self.n_x = input_dim
        self.n_h = hidden_dim
        self.n_y = output_dim
        self.n_z = context_dim
        self.m = 9  #dimension of basis function vector (polynomial features) for 3 context features


        # reset gate components
        self.linear_reset_w1 = nn.Linear(self.n_x * self.m, self.n_h, bias=True)
        self.linear_reset_r1 = nn.Linear(self.n_h, self.n_h, bias=True)


        self.linear_reset_w2 = nn.Linear(self.n_x * self.m, self.n_h, bias=True)
        self.linear_reset_r2 = nn.Linear(self.n_h, self.n_h, bias=True)
        self.activation_1 = nn.Sigmoid()

        # update gate components
        self.linear_gate_w3 = nn.Linear(self.n_x * self.m, self.n_h, bias=True)
        self.linear_gate_r3 = nn.Linear(self.n_h, self.n_h, bias=True)
        self.activation_2 = nn.Sigmoid()

        self.activation_3 = nn.Tanh()

        #output
        self.linear_output = nn.Linear(self.n_h, self.n_y, bias=True)


    def reset_gate(self, xg, h):  #xg is the kronecker product of x and  basis function G(z)
        x_1 = self.linear_reset_w1(xg)
        h_1 = self.linear_reset_r1(h)
        # gate update
        r = self.activation_1(x_1 + h_1)
        return r

    def update_gate(self, xg, h):
        x_2 = self.linear_reset_w2(xg)
        h_2 = self.linear_reset_r2(h)
        s = self.activation_2( h_2 + x_2)
        return s


    def update_component(self, xg, h, r):
        x_3 = self.linear_gate_w3(xg)
        h_3 = r * self.linear_gate_r3(h)
        h_tilda = self.activation_3(x_3+h_3)
        return h_tilda


    def compute_output(self,h):
        y_pred = self.linear_output(h)
        return y_pred


    def cell_forward(self, x, h, G):

        """
        Implements a single forward step of the Context GRU-cell

        Input Arguments:
            x (mini-batch): input x at time step t , (n,n_x) : (batch_size, input_dim)
            h : hidden state at time step t-1, (n,n_h) : (batch_size, hidden_dim)
            G : vector of basis funcitons (m,n)

        Returns:
            h_new: hidden state at time step t, (n,n_h)

        """

        # kronecker product of x and G(zt)
        n = x.shape[0]
        xg = torch.zeros(n,self.n_x*self.m).to(device)

        for i in range(n):

            xg[i,:] = torch.kron(x[i,:],G[:,i])


        # Equation 1. reset gate vector
        r = self.reset_gate(xg, h)

        # Equation 2: the update gate - the shared update gate vector z
        s = self.update_gate(xg, h)

        # Equation 3: The almost output component
        h_tilda = self.update_component(xg,h,r)

        # Equation 4: the new hidden state
        h_new = (1-s) * h_tilda  + s * h

        #output

        y_pred = self.compute_output(h)

        return h_new, y_pred


    def forward(self, x, z):

        """
        Implement the forward propagation of the recurrent neural network

        Input Arguments:
        x (mini_batch): primary input for every time-step in mini-batches of shape (n, T, n_x)
        z (mini_batch): context input for every time-step in mini-batches of shape (n,T,n_z)


        Returns:
            h -- Hidden states for every time-step, numpy array of shape (n, T, n_h)
            y_pred -- Predictions for every time-step, numpy array of shape (n, T, n_y),
            here T is 1 for Seq to Vec RNN
        """

        # Retrieve dimensions from shapes of x
        #print(x.shape)
        #print(z.shape)
        n,T,n_x = x.shape
        n_y = self.n_y
        n_h = self.n_h
        n_z = self.n_z



        # initialize "h"

        h = self.init_hidden(n,T,n_h)

        #y_pred = np.zeros((m,T_x,n_y))
        #y_pred is single value for one sample, m=1

        #basis function vector
        G = self.apply_basis(z[:,0,:])  #G: size of (n,m)

        #for initial time step the hidden state is 0
        h_temp = h.clone()
        h_init = h_temp[:,0,:]
        h_curr, y_curr = self.cell_forward(x[:,0,:],h_init,torch.t(G))

        # loop over all time-steps
        for t in range(1,T):

            #compute the vector of basis functions

            G = self.apply_basis(z[:,t,:])  #G: size of (n,m)

            # Update next hidden state
            # ignore yt_pred for seq to vector
            h[:,t,:]= h_curr
            h_temp = h.clone()
            h_prev = h_temp[:,t,:]  #h_prev: (n,n_h)
            h_curr, y_curr = self.cell_forward(x[:,t,:],h_prev, torch.t(G))

            #y_pred[t,:] = yt_pred


        #compute the predicted output from the last cell i.e at last time step T
        y_pred = torch.zeros(n,1,1,device = 'cuda:0')

        #get the value of y_pred from the last cell
        y_pred[:,0,:] = y_curr

        #print(y_pred.shape)



        return h, y_pred


    def init_hidden(self, n:int,T:int, n_h:int):
        #initialise the hidden state
        #n : batch-size
        #T : Input sequence length
        #returns h of size (n,T,n_h)
        return torch.zeros(n,T,n_h,device = 'cuda:0')


    def apply_basis(self,zt):
        '''
        apply the basis function: polynomial degree 2
        [z0, z1, z2, z0z0, z0z1, z0z2....]
        input arguments:
            zt: context vector (n,n_z) for mini-batch of size n and n_z context dim
        Returns:
            G : tensor of basis functions, (m,n)

        for 3 context features m = 9
        '''

        #poly = PolynomialFeatures(2, include_bias=False, interaction_only=True)
        poly = PolynomialFeatures(2, include_bias=False)
        G = torch.tensor(poly.fit_transform(zt.cpu().numpy())).to(device) #fit_transform returns nd array
        #print(G.shape)



        return G




Using cuda device


In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.weights = []      #used for visualising weights
        self.settings = []     #saving settings for visualising weights
        self.inputs = []       #saving input for visualising

    def train_step(self, x, y, z):

       # with profiler.record_function("TRAIN STEP FUNCTION"):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        h, yhat = self.model(x, z)


        # Computes loss
        loss = self.loss_fn(y, yhat)

        #with profiler.record_function("LOSS_BACKWARD"):
        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss

        return loss.item()

    def train(self, train_loader, val_loader, batch_size, n_epochs=50, np_features=1, nc_features=1):
        '''
        np_features = # primary input features
        nc_features = # context input features
        '''
        #model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        times = []
        for epoch in range(1, n_epochs + 1):

            start_epoch = time.time()

            batch_losses = []
            batch_count = 0
            for x_batch, z_batch, y_batch in train_loader:
                batch_count += 1
                x_batch = x_batch.view([batch_size,-1, np_features]).to(device)
                y_batch = y_batch.to(device)
                z_batch = z_batch.view([batch_size,-1, nc_features]).to(device)

                #with profiler.profile(with_stack=True, profile_memory=True) as prof:
                loss = self.train_step(x_batch, y_batch, z_batch)
                #print(prof.key_averages(group_by_stack_n=5).table(sort_by = 'self_cpu_time_total', row_limit = 5))

                batch_losses.append(loss)

            # if (epoch % 10 == 0):
            #         #if (batch_count % 200 == 0):
            #             #save the model weights for each batch for analysis
            #             #self.save_model(self.model, batch_count, str(z_batch[-1,:,:].detach().cpu().numpy()))
            #     for param_tensor in model.state_dict():
            #         if (param_tensor == 'linear_reset_w1.weight'):
            #             param_val = model.state_dict()[param_tensor].cpu().numpy().tolist()
            #             self.weights.append(param_val)
            #             self.settings.append(z_batch[-1,:,:].detach().cpu().numpy().tolist())
            # #self.model.to(device)


            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)




            with torch.no_grad():
                batch_val_losses = []
                for x_val, z_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, np_features]).to(device, non_blocking=True)
                    y_val = y_val.to(device)
                    z_val = z_val.view([batch_size, -1, nc_features]).to(device,non_blocking=True)
                    self.model.eval()

                    # with profiler.profile(with_stack=True, profile_memory=True) as prof:
                    h,yhat = self.model(x_val, z_val)
                    # print(prof.key_averages(group_by_stack_n=5).table(sort_by = 'self_cpu_time_total', row_limit = 5))

                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch % 5 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )



            torch.cuda.synchronize()
            end_epoch = time.time()
            elapsed = end_epoch - start_epoch
            times.append(elapsed)

        total_time = sum(times)
        avg_time = sum(times)/n_epochs

        print(f"Average Training time: {avg_time:.4f} s for epochs {n_epochs}")

        print(f"Total Training time: {total_time:.4f} s for epochs {n_epochs}")


        #torch.save(self.model.state_dict(), model_path)

        return validation_loss  #this will be used by otuna to optimize

    def evaluate(self, test_loader, batch_size=1, np_features=1, nc_features = 1):
            with torch.no_grad():
                predictions = []
                values = []
                for x_test, z_test, y_test in test_loader:

                    x_test = x_test.view([batch_size,-1, np_features]).to(device, non_blocking=True)
                    y_test = y_test.to(device)
                    z_test = z_test.view([batch_size,-1, nc_features]).to(device, non_blocking=True)
                    self.model.eval()
                    h,yhat = self.model(x_test, z_test)
                    predictions.append(yhat.detach().cpu().numpy())
                    values.append(y_test.detach().cpu().numpy())

            return predictions, values

    def plot_losses(self):
            plt.plot(self.train_losses, label="Training loss")
            plt.plot(self.val_losses, label="Validation loss")
            plt.legend()
            plt.title("Losses")
            plt.xlabel("Epochs")
            plt.ylabel("Loss(MSE)")
            plt.show()
            plt.close()

    def save_model(self, model, batch_id, settings_val):

        # path = r'/home/rashmi/PythonProjects/codes/TURBOFAN_MODELS/savedmodels/'
        # #save model
        # file_name = 'FD002_Params.txt'
        # file_path = os.path.join(path,file_name)
        # f = open(file_path, 'a')
        # f.write('Batch'+ str(batch_id)+'\n')
        # f.write('-------\n')
        # for param_tensor in model.state_dict():
        #     param_val = model.state_dict()[param_tensor].cpu().numpy().tolist()
        #     f.write(param_tensor + "\t" + str(param_val))
        #     f.write('\n---------------\n')
        # f.write('Settings\n')
        # f.write(settings_val + '\n')
        # f.write('---------------\n')
        # f.write('\n')

        # f.close()


    def visualise_weights(self):

#         col = ['r','b','g']
#         nrows = len(self.weights)
#         for i in range(nrows):
#             wtmatrix = np.array(self.weights[i])
#             print(wtmatrix.shape)
#             fig = plt.figure()
#             print(wtmatrix[0:9, 0:9])
#             #plt.imshow(wtmatrix[0:9, 0:9])
#             sns.heatmap(wtmatrix[0:10, 0:10])

#         fig = plt.figure()
#         for i in range(nrows):
#             wtmatrix = np.array(self.weights[i])
#             plt.plot(wtmatrix[0:10],wtmatrix [0:10],color = col[i],marker = '.')

        return self.weights, self.settings, self.inputs





## 1. Data Loading

In [ ]:
#data load function

def dataload(filename):

    df = pd.read_csv(filename)
    return df

In [ ]:
#load the combined sensor and weather data
#207 is number of sensors

df_w = []
#path3 =       #define path
for i in range(0,207):
    filename = 'sw_' + str(i+1)
    df_w.append(dataload(path3+filename))


In [ ]:
for i in range(len(df_w)):
    df_w[i].drop(columns = [df_w[i].columns[0]], inplace = True)

In [ ]:
df_w[0].head()

,Year,Month,Day,Hour,Weekday,StationId,AvgSpeed,AvgSpeed1,AvgSpeed2,AvgSpeed3,AvgSpeed4,AvgSpeed5,AvgSpeed6,AvgSpeed7,Precipitation,Temp,WindSpeed
0,2012.0,3.0,8.0,0.0,3.0,773869.0,69.3,69.4,69.6,69.8,69.3,70.5,69.6,68.9,0.0,5.2,2.15
1,2012.0,3.0,8.0,0.0,3.0,773869.0,68.4,68.4,69.8,69.4,68.9,68.1,70.0,67.5,0.0,5.2,2.15
2,2012.0,3.0,8.0,0.0,3.0,773869.0,69.1,68.8,66.7,67.6,67.2,68.7,66.7,68.7,0.0,5.2,2.15
3,2012.0,3.0,8.0,0.0,3.0,773869.0,69.2,67.8,67.6,67.6,68.8,71.2,65.7,69.1,0.0,5.2,2.15
4,2012.0,3.0,8.0,0.0,3.0,773869.0,68.3,69.0,65.1,67.9,68.6,71.3,68.2,67.2,0.0,5.2,2.15


## 2. Data Prepreprocessing

### a) Smoothing - moving average

In [ ]:
#Trailing moving average
#trail_ma(t) = mean(obs(t-2), obs(t-1), obs(t))

def moving_average(x, w):
    #x: time series
    #w: sliding window size
    return np.convolve(x, np.ones(w), 'valid') / w

### b) Normalisation - Transform and Inverse transform

In [ ]:
def data_transform(data,option ='minmax'):
#data is numpy array
#option is set to std for standardization or minmax

    n = data.shape[0]

    if option == 'std' :

        #perform standardization of data
        miu = np.mean(data,axis = 0)
        sigma = np.std(data,axis=0,dtype=float)
        temp_data = data-np.tile(miu,(n,1))
        std_data = np.divide(temp_data,np.tile(sigma,(n,1)))

        return std_data, miu, sigma

    elif option == 'minmax':

        #perform min-max normalization
        max_val = np.max(data,0)
        #print(max_val)
        min_val = np.min(data,0)
        #print(min_val)
        rng = max_val-min_val
        norm_data = np.divide(data - np.tile(min_val,(n,1)),np.tile(rng,(n,1)))

        return norm_data, min_val, rng

In [ ]:
## Inverse transform the target/outout data from normalized to original

def inv_trans(data,option,param1, param2):
    # apply inverse of standardization or normalization for 1-D column/row vector
    # option: standard or minmax normalization
    # params:list of parameters applied while normalization
    # data is 1-D column vector
#     print(data)
#     print(param1)
#     print(param2)

    if option == "std":
         #perform standardization of data
        miu = param1
        sigma = param2
        inv_data = data*sigma + miu

        return inv_data

    else : #MinMax normalization

        #perform min-max normalization
        min_val = param1
        rng = param2
        inv_data = data*rng+min_val
        return inv_data

### Minmax normalisation

In [ ]:
df_arr = []  #list of arrays -207
min_val = []  #list of minimum value used for normalization
rng = []  #list of range values used for normalization
for i in range(len(df_w)):
    arr = np.array(df_w[i])
    norm_arr, m_val, r_val = data_transform(arr[:,6:17])  #Year, Month, ..., StationID are not normalised
    final_arr = np.concatenate((arr[:,0:6], norm_arr), axis=1)
    df_arr.append(final_arr)
    min_val.append(m_val)
    rng.append(r_val)


In [ ]:
df_arr[0].shape

(32724, 17)

## 3. Data Preparation

In [ ]:
df_w[0].columns

Index(['Year', 'Month', 'Day', 'Hour', 'Weekday', 'StationId', 'AvgSpeed',
       'AvgSpeed1', 'AvgSpeed2', 'AvgSpeed3', 'AvgSpeed4', 'AvgSpeed5',
       'AvgSpeed6', 'AvgSpeed7', 'Precipitation', 'Temp', 'WindSpeed'],
      dtype='object')

In [ ]:
# Prepare the data for RNN model such that the data is presented as (num_samples,seq_length,num_features)

def data_preparation(data,n_past,n_future):
    '''
    input:
        data : input data
        n_past : number of past steps to be used for prediction
        n_future :  number of steps ahead

    returns:
        context input (Z): 'Precipitation','Temp', 'WindSpeed'
        primary input (X): 'AvgSpeed', AvgSpeed1..AvgSpeed7'
        ouput/target (Y): 'AvgSpeed' at time step t
        Additional Information (U) : Year, Month, ...StationId
    '''

    n,m = data.shape


    k = n_future
    t = n_past


    input_data = []
    periodic_input = []
    output_data = []
    context_data = []
    info_data = []
    comb_input = []


    for i in range(t, (n-k+1)):

        #info_data.append(data[i-t:i,0:6])  #first 6 columns are additional information
        input_data.append(data[i-t:i,6:7])  #Avg Speed
        context_data.append(data[i-t:i,14:17]) # weather data
        periodic_input.append(data[i+k-t:i+k,7:14])
        output_data.append([data[i+k-1:i+k,6]])  #Target AvgSpeed
        info_data.append(data[i+k-1:i+k,0:6])  #first 6 columns are additional information

#     print(np.array(input_data).shape)
#     print(np.array(periodic_input).shape)
#     print(np.array(context_data).shape)
#     print(np.array(output_data).shape)

    #combine input_data and periodic_data
    X = np.concatenate((np.array(input_data), np.array(periodic_input)), axis = 2)
    Y = np.array(output_data)
    Z = np.array(context_data)
    U = np.array(info_data)




    #print(n,m)
#    print(X.shape)
#     print(Y.shape)
#     print(Z.shape)


    return X, Y, Z, U


### a) Data preparation (Minmax normalised)

In [ ]:
'''
prediction n_future = step ahead with n_past samples
n_future = 12 steps (next hour)
n_past = 12 steps
prediction of one hour ahead using lone hour past samples.
Example:  8:00, 8:05, ...9:00 past data (same day) and 9:00, 9:05...10:00 past data (last week
during same time slot and day as target = periodic data) --> predicting 10:00 traffic avg speed
'''

'''
Divide the data into 70:10:20 Train:Validation:Test
'''

seq_len = 12 #sequence length for lstm  #n_past
k = 12  #k steps ahead prediction (one hour ahead prediction)


#prepare the data for each sensor separately
#X, Y, Z, U are 3 dim (num_samples,Sequence length, num_features)

df_X = []  #list for saving input data (X) for each sensor (StationID)
df_Y = []
df_Z = []
df_U = []

for i in range(len(df_arr)):
    X, Y, Z, U = data_preparation(df_arr[i],seq_len,k)
    df_X.append(X)
    df_Y.append(Y)
    df_Z.append(Z)
    df_U.append(U)


# print(X_train.shape)
# print(X_val.shape)



In [ ]:
print(df_X[0].shape)
print(df_Y[0].shape)
print(df_Z[0].shape)
print(df_U[0].shape)

(32701, 12, 8)
(32701, 1, 1)
(32701, 12, 3)
(32701, 1, 6)


## 4. Loading data into PyTorch Dataloaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def dataloader_pytorch (train_data, val_data, batch_size = 128):

    #train_data and val_data is tuple with X, Y, Z

    X_train = train_data[0]
    Y_train = train_data[1]
    Z_train = train_data[2]

    X_val = val_data[0]
    Y_val = val_data[1]
    Z_val = val_data[2]

    #transform the arrays into torch tensors
    train_features = torch.Tensor(X_train)  #drop unit_number and test_cycles
    train_targets = torch.Tensor(Y_train)
    train_cx_features = torch.Tensor(Z_train)

    val_features = torch.Tensor(X_val)
    val_targets = torch.Tensor(Y_val)
    val_cx_features = torch.Tensor(Z_val)


    train = TensorDataset(train_features,train_cx_features, train_targets)
    val = TensorDataset(val_features, val_cx_features,val_targets)


    train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

    examples = iter(train_loader)
    samples,context,targets = examples.next()
    print(samples.shape, context.shape,targets.shape)

    return train_loader, val_loader

#### Seprate the train val and test data

In [ ]:
from sklearn.model_selection import train_test_split


batch_size = 128
# train_loader = []
# val_loader = []

test_data = []

#for first sensor data
X = df_X[0].copy()
Y = df_Y[0].copy()
Z = df_Z[0].copy()
U = df_U[0].copy()

# split the data in 70:10:20 for train:valid:test dataset
train_size=0.7

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, Y_train, Y_rem, Z_train, Z_rem, U_train, U_rem = train_test_split(X,Y,Z,U, train_size=0.7)

test_size = 0.6
X_val, X_test, Y_val, Y_test, Z_val, Z_test, U_val, U_test = train_test_split(X_rem,Y_rem, Z_rem, U_rem, test_size = 0.6)


test_data.append((X_test,Y_test,Z_test,U_test))


#for remaining sensor data
for i in range(1, len(df_X)):

    #prepared data: df_X, df_Y, df_Z, df_U

    X = df_X[i].copy()
    Y = df_Y[i].copy()
    Z = df_Z[i].copy()
    U = df_U[i].copy()

    # split the data in 70:10:20 for train:valid:test dataset
    train_size=0.7

    # In the first step we will split the data in training and remaining dataset
    X_train1, X_rem1, Y_train1, Y_rem1, Z_train1, Z_rem1, U_train1, U_rem1 = train_test_split(X,Y,Z,U, train_size=0.7)


    test_size = 0.6
    X_val1, X_test1, Y_val1, Y_test1, Z_val1, Z_test1, U_val1, U_test1 = train_test_split(X_rem,Y_rem, Z_rem, U_rem, test_size = 0.6)

#     print(X_train.shape), print(Y_train.shape), print(Z_train.shape)
#     print(X_val.shape), print(Y_val.shape), print(Z_val.shape)
#     print(X_test.shape), print(Y_test.shape), print(Z_test.shape)

#     train_data.append = (X_train, Y_train, Z_train)
#     val_data.append = (X_val, Y_val, Z_val)

    #combine the X, Y, Z from all the sensors for training and validation
    X_train = np.concatenate((X_train,X_train1), axis = 0)
    Y_train = np.concatenate((Y_train,Y_train1), axis = 0)
    Z_train = np.concatenate((Z_train,Z_train1), axis = 0)

    X_val = np.concatenate((X_val,X_val1), axis = 0)
    Y_val = np.concatenate((Y_val,Y_val1), axis = 0)
    Z_val = np.concatenate((Z_val,Z_val1), axis = 0)



    test_data.append((X_test1,Y_test1,Z_test1,U_test1))

    #train_l, val_l = dataloader_pytorch (train_data, val_data,batch_size)
    #train_loader.append(train_l)
    #val_loader.append(val_l)

train_data = (X_train,Y_train,Z_train)
val_data = (X_val, Y_val, Z_val)
train_loader, val_loader = dataloader_pytorch (train_data, val_data,batch_size)
#train_loader.append(train_l)
#val_loader.append(val_l)


torch.Size([128, 12, 8]) torch.Size([128, 12, 3]) torch.Size([128, 1, 1])


## 5. Hyperparameter Optimization using Optuna

In [ ]:
#Hyperparameter optimization with optuna


input_dim = df_X[0].shape[2]
output_dim = df_Y[0].shape[2]
context_dim = df_Z[0].shape[2]

weight_decay = 1e-6
dropout = 0.1
n_epochs = 50
batch_size = 128

#dimensions of fully connected layer
#fc_dim = 5

def objective(trial):

    params1 = {
              'input_dim':input_dim,
              'output_dim': output_dim,
              'context_dim': context_dim,
              'hidden_dim':trial.suggest_int('hidden_size',10,30,5),
              #'fc_dim': trial.suggest_int('fc_size',5,30,5),
              }

    params2 = {
                'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
                'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }

    #model = get_model('gru',params1)
    model = ContextGRU(input_dim, params1['hidden_dim'], output_dim, context_dim, params1['fc_dim'])
    model = model.to(device)


    loss_fn = nn.MSELoss(reduction="mean")
    #optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"], weight_decay=weight_decay)
    optimizer = getattr(optim, params2['optimizer'])(model.parameters(), lr= params2['learning_rate'], weight_decay=weight_decay)
    opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
    train_loss = opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, np_features=input_dim, nc_features=context_dim)
    opt.plot_losses()


    return train_loss

##----------------------------------------------------------------------------------------------
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))


## 6. Training

In [ ]:
input_dim = df_X[0].shape[2]
output_dim = df_Y[0].shape[2]
context_dim = df_Z[0].shape[2]

hidden_dim = 15
layer_dim = 1
batch_size = 128
dropout = 0.2
n_epochs = 50
learning_rate = 0.001
weight_decay = 1e-6

model = ContextGRU(input_dim, hidden_dim, output_dim, context_dim)

model = model.to(device)

params_list = model.parameters()

loss_fn = nn.MSELoss(reduction="mean")

#optimizer = optim.Adam(params_list,lr=learning_rate, weight_decay=weight_decay)
optimizer = optim.RMSprop(params_list, lr=learning_rate, alpha=0.99, eps=1e-08, weight_decay=weight_decay)
#optimizer = optim.SGD(params_list,lr=learning_rate, weight_decay=weight_decay)
opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, np_features=input_dim, nc_features=context_dim)
opt.plot_losses()
#opt.visualise_weights()

## Analysis of model weights

In [ ]:
# # Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", str(model.state_dict()[param_tensor].size()))

wts, sets, ips = opt.visualise_weights()

l = len(wts)
wtmatrix = np.array(wts[l-1])
fig = plt.figure(figsize=(15, 8))

for i in range(6) :  #nx = 6 and m = 9

    plt.subplot(2,3,i+1)

    sns.heatmap(wtmatrix[:,i*9:i*9+9], cmap = 'Spectral')





## 7. Testing

In [ ]:
 #test_data-list of tuples (X_test,Y_test,Z_test,U_test)

result_rmse = []  #list of rmse for each sensor id
result_mae = []
result_r2 = []
s = 207 #number of sensors


for i in range(0, s):

#for i in range(30,31):

    (X_test, Y_test, Z_test, U_test) = test_data[i]


    test_features = torch.Tensor(X_test)
    test_targets = torch.Tensor(Y_test)
    test_cx_features = torch.Tensor(Z_test)

    test = TensorDataset(test_features,test_cx_features, test_targets)
    #test_loader = DataLoader(test, batch_size=X_test.shape[0], shuffle=False, drop_last=True)
    test_loader_one = DataLoader(test, batch_size=1, shuffle=False)


    predictions, values = opt.evaluate(test_loader_one, batch_size=1, np_features=input_dim, nc_features = context_dim)
    #flatten the multi-dimension array to 1-D array
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()

    #Apply inverse transform
    #reshape vals and preds as inverse transform accepts 2-D array


    #parameters from train data statistics
    p1 = min_val[i]
    par1 = p1[6]   #parameter for 'AvgSpeed'  (mean/miu)
    p2 = rng[i]
    par2 = p2[6]  #parameter for 'AvgSpeed'  (range/sigma)


    #denormalise the target
    target_val = inv_trans(np.reshape(vals,(len(vals),1)), "minmax",par1,par2)
    pred_val = inv_trans(np.reshape(preds,(len(preds),1)),"minmax",par1,par2)

    #plot_results(i+1, target_val[0:200],pred_val[0:200])

    result_metrics = calculate_metrics(target_val, pred_val)  #result_metrics is a dictionary

    result_rmse.append(result_metrics['rmse'])
    result_r2.append(result_metrics['r2'])
    result_mae.append(result_metrics['mae'])


#calculate average and std of rmse,and score

avg_rmse = np.mean(result_rmse)
std_rmse = np.std(result_rmse)

avg_mae = np.mean(result_mae)
std_mae = np.std(result_mae)

print(f"[Average RMSE: {avg_rmse:.4f}\t Std RMSE: {std_rmse:.4f}]")
print(f"[Average MAE: {avg_mae:.4f}\t Std MAE: {std_mae:.4f}]")


min_rmse = min(result_rmse)
max_rmse = max(result_rmse)
min_mae = min(result_mae)
max_mae = max(result_mae)

print(f"[Sensor #: {result_rmse.index(min_rmse)+1}\t Min RMSE: {min_rmse:.4f}]")
print(f"[Sensor #: {result_rmse.index(max_rmse)+1}\t Max RMSE: {max_rmse:.4f}]")
print(f"[Sensor #: {result_mae.index(min_mae)+1}\t Min MAE: {min_mae:.4f}]")
print(f"[Sensor #: {result_mae.index(max_mae)+1}\t Max MAE: {max_mae:.4f}]")


In [ ]:
info = U_test[:,0,:]
info.shape

(5887, 6)

In [ ]:
#combine info,target, and perd_val
data_result = np.concatenate((info,target_val,pred_val), axis =1)

In [ ]:
df_result = pd.DataFrame(data_result, columns =['Year', 'Month', 'Day', 'Hour', 'Weekday', 'StationId', 'Target', 'Predicted'])

In [ ]:
df_result.head()

,Year,Month,Day,Hour,Weekday,StationId,Target,Predicted
0,2012.0,3.0,20.0,4.0,1.0,773869.0,67.499992,67.720848
1,2012.0,4.0,16.0,19.0,0.0,773869.0,68.099998,69.193779
2,2012.0,6.0,15.0,14.0,4.0,773869.0,66.899994,67.107353
3,2012.0,5.0,11.0,22.0,4.0,773869.0,70.000000,68.017006
4,2012.0,4.0,19.0,8.0,3.0,773869.0,67.899994,66.763290


In [ ]:
df_select_mon = df_result[df_result['Month'] == 5.0]

In [ ]:
df_select_day = df_select_mon[df_select_mon['Day'] == 15.0]

In [ ]:
df_select_day.head()

,Year,Month,Day,Hour,Weekday,StationId,Target,Predicted
178,2012.0,5.0,15.0,5.0,1.0,773869.0,65.299995,65.338074
348,2012.0,5.0,15.0,3.0,1.0,773869.0,15.099998,12.385771
398,2012.0,5.0,15.0,9.0,1.0,773869.0,46.100002,66.639671
692,2012.0,5.0,15.0,12.0,1.0,773869.0,65.000000,62.158409
893,2012.0,5.0,15.0,9.0,1.0,773869.0,66.399994,65.154655


In [ ]:
plot_data = df_select_day.sort_values(by=['Hour'])

In [ ]:
plot_data.reset_index(inplace=True)


In [ ]:
plot_data.head()

,index,Year,Month,Day,Hour,Weekday,StationId,Target,Predicted
0,5572,2012.0,5.0,15.0,1.0,1.0,773869.0,66.799995,65.273506
1,4190,2012.0,5.0,15.0,1.0,1.0,773869.0,61.099998,62.943649
2,5303,2012.0,5.0,15.0,2.0,1.0,773869.0,62.299999,64.882828
3,4358,2012.0,5.0,15.0,2.0,1.0,773869.0,64.000000,67.650780
4,1714,2012.0,5.0,15.0,2.0,1.0,773869.0,65.699997,66.177315


In [ ]:
plot_data.drop(columns=['index'], inplace = True)

In [ ]:
plot_data.drop_duplicates(subset=['Hour'], inplace = True)

In [ ]:
plot_data.head()

,Year,Month,Day,Hour,Weekday,StationId,Target,Predicted
0,2012.0,5.0,15.0,1.0,1.0,773869.0,66.799995,65.273506
2,2012.0,5.0,15.0,2.0,1.0,773869.0,62.299999,64.882828
5,2012.0,5.0,15.0,3.0,1.0,773869.0,15.099998,12.385771
8,2012.0,5.0,15.0,4.0,1.0,773869.0,65.699997,66.747032
11,2012.0,5.0,15.0,5.0,1.0,773869.0,66.699997,68.316055


In [ ]:
date = '15-05-2012'
plot_results(plot_data,date)

In [ ]:
#Plot the results Engine Unit wise
def plot_results(plot_data, date):

    plot_data.reset_index(inplace=True)
    #print(plot_data)

    target_val = plot_data['Target']
    pred_val = plot_data['Predicted']

    #fig = plt.figure(figsize=(14, 4))
    fig = plt.figure()
    ax = plt.subplot(1,2,1)
    #plt.subplot(1,2,1)
    plt.subplot(1,2,1).set_title(date, fontsize= 10)
    ax.plot(target_val,'r')
    ax.plot(pred_val,'b')
    plt.xlabel('Time')
    plt.ylabel('Avg. Speed (Mph)')
    plt.legend(['Actual Avg. Speed','Predicted Avg. Speed'])

    ticks = list(range(0,len(plot_data)))
    tick_labels = plot_data['Hour']
    ax.set_xticks(ticks[::3])
    ax.set_xticklabels(tick_labels[::3])
#     ax.set_xticks(ticks)
#     ax.set_xticklabels(tick_labels)
    ax.tick_params(axis='x')
    plt.savefig('traffic_result.eps', format = eps, dpi=1000)


### Calculate Error Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(actual, predicted):

    return {'mae' : mean_absolute_error(actual,predicted),
            'rmse' : mean_squared_error(actual,predicted) ** 0.5,
            'r2' : r2_score(actual,predicted)}

# result_metrics = calculate_metrics(target_val, pred_val)
# print(result_metrics)